# MD simulation of a protein-ligand system using OpenMM.

This is part 3 of a three-part tutorial on molecular dynamics simulations of biomolecular systems prepared for *CompBioAsia* 2025.

Part 1 covered the process of preparing a molecular system for MD simulation.

Part 2 dealt with actually running MD simulations, using the **AMBER** package.

This third part looks at another MD package - **OpenMM**.

## Prerequisites
Assuming you have started this Notebook using the `run_notebook.sh` script in this folder, your Python environment should be complete.

## Background
[OpenMM](https://openmm.org/) is a powerful Python-focussed package for molecular dynamics simulations. Click on the link to learn lots about it, but for the present purposes the key learning objective is to become aware of a rather diferent approach to running MD simulations than using the more 'old-fashioned' (but still very widely used and very powerful) command-line focussed tools like **AMBER** and **GROMACS** for example.


**Authors**:
This tutorial is adapted from CCPBioSim's [BioSim analysis workshop](https://github.com/CCPBioSim/BioSim-analysis-workshop).

*Updates*: Charlie Laughton (charles.laughton@nottingham.ac.uk)

## Part 1: Prepare your system for MD with OpenMM

OpenMM has its own tools for system preparation (fixing-up starting structures, paramaterizing systems according to chosen force fields, etc.) but it can also accept systems that have been prepared by other programs such as AMBER. 

As that's what we did in the first of these tutorials, we will use that data. Copies of `abl_ligand.prmtop` and `abl_ligand.inpcrd` are included in this folder.

### 1.1 Make an OpenMM `System`

Throughout these tutorials we have used the word *system* quite frequently but quite loosely - in general it has meant "everything that's going to be simulated". `OpenMM` as a Python package has it's own very specific `system` object - see [here](http://docs.openmm.org/latest/api-python/generated/openmm.openmm.System.html#openmm.openmm.System) for its definition. Our first task will be to construct one of these, using the data from our AMBER-generated input files.

First we load the required packages:

In [ ]:
import openmm.app as app 
import openmm as omm
import openmm.unit as unit
import sys

Next we load our AMBER-generated data files:

In [ ]:
prmtop = app.AmberPrmtopFile('abl_ligand.prmtop')
inpcrd = app.AmberInpcrdFile('abl_ligand.inpcrd')

Now we generate the `system`. If you looked at the link above, you will have seen that OpenMM `system`s contain **all** the information about the forces that will be present in any simulation that is done. So we have to define all of these, which includes some aspects that will not yet be present from the AMBER files, such as how non-bonded interactions will be handled, and how constant pressure conditions will be managed.

In [ ]:
system = prmtop.createSystem(nonbondedMethod=app.PME, # Particle-mesh Ewald method
                             nonbondedCutoff=1*unit.nanometer,
                             constraints=app.HBonds) # Lengths of bonds to hydrogena atoms 
                                                     # are constrained


barostat = omm.MonteCarloBarostat(
        1.0 * unit.bar, # Target pressure
        300.0 * unit.kelvin, # Target temperature
        25) # How many time steps between pressure adjustment

system.addForce(barostat) # Add the barostat to the system
print('system object created')

### 1.2 Make an OpenMM `Simulation`

AN OpenMM `system` defines things that are invariant - what atoms are present and what forces will act on them. Interestingly though, a `system` does not describe the *topology* - how the atoms are connected to each other, which is another thing that in a molecular mechanics-based simulation such as this must also be invariant. 

OpenMM then has another object, a `simulation`, that represents a currently-running simulation of this system - see [here](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) for details. That includes how it will be run (on what hardware, and using what algorithms), and everything that may change over time, such as the coordinates and velocities of the atoms.

In [ ]:
# Define the integrator to use:
integrator = omm.LangevinIntegrator(
        300*unit.kelvin,  # The temperature of the simulation
        1/unit.picosecond, # The temperature coupling parameter
        0.002*unit.picoseconds) # The simulation timestep

# Create the simulation object:
simulation = app.Simulation(prmtop.topology, system, integrator)

# Set the initial positions of the atoms and the initial periodic box size:
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
print('simulation object created')

## Part 2: Molecular Simulation

### 2.1 Energy minimisation

Before running MD, it;s always advisable to make sure the initially-build system is free from serious structural defects, such as atom clashes or bad geometry, by running an energy minimisation. Using OpenMM, this is very simple:

In [ ]:
simulation.minimizeEnergy()
print('energy minimization complete')

### 2.2 Molecular dynamics simulation
With the simulation now energy minimised, we can start dynamics.

#### 2.2.1 Add reporters
You will have noticed that the energy minimisation step was very "quiet" - you got no information about what happened. So before we run the MD simulation, we need to attach `reporters` to our `simulation`, so that data we are interested in gets captured. Here we use three reporters:

1. One that writes the step number to the screen - just so we know things are progressing
2. One that writes interesting attributes of the simulation to a log file - things like the current temperature, potential energy, etc.
3. One that writes the current coordinates of the system to a *trajectory file*.

We ask each of these reporters to provide output every 100 simulation time steps.

In [ ]:
from parmed.openmm import StateDataReporter
from mdtraj.reporters import NetCDFReporter

simulation.reporters.append(
        StateDataReporter(sys.stdout, 100, step=True, potentialEnergy=False,
                          kineticEnergy=False, totalEnergy=False,
                          density=False, temperature=False)
)

simulation.reporters.append(
        StateDataReporter('abl_ligand.log', 100, density=True) # Report density as well as default
                                                               # parameters such as temperature
)

simulation.reporters.append(
        NetCDFReporter('abl_ligand.nc', 100)
)
print('reporters added')

#### 2.2.2 Run MD

Now we can run some MD. We just need to decide the total number of timsteps to run. Here we use 10,000, so remembering that we set the integrator timestep to be 0.002 picoseconds, that equates to 20ps of simulation in total.

In [ ]:
# Run dynamics
print('Running dynamics')
simulation.step(10000)
print('simulation complete')

## 3. Analysis of the simulation data

Let's have a quick look at the data we have generated and saved to our two output files - the log file `abl_ligand.log` and the trajectory file `zabl_ligand.nc`. 

### 3.1 Visualization of the trajectory

In [ ]:
import mdtraj as mdt
import nglview as nv

In [ ]:
traj = mdt.load('abl_ligand.nc', top='abl_ligand.prmtop')
print(traj)
view = nv.show_mdtraj(traj)
view

The visualization does not show the water molecules, so the solute can be seen more clearly. If you refer back to earlier notebooks in this series, you should be able to work out how to change this behaviour if you prefer.

#### 3.2 Graphical analysis of simulation parameters

Though it's always nice to visualize a trajectory, normally the most valuable data from a simulation is in numerical properties such as those captured in the log file. Here we plot the potential enerhy, temperature, and density of the simulation as a function of time.

We will use `pandas` and `matplotlib` for this:

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

reporters = pd.read_csv('abl_ligand.log')
reporters

Defining a function makes the plotting process a bit neater:

In [ ]:
def plot_reporters(reportfilename):

  reporters = pd.read_csv(reportfilename)

  fig, axs = plt.subplots(1, 3, figsize=(14, 3))
  axs[0].plot(reporters['Time (ps)'], reporters['Potential Energy (kilocalorie/mole)'])
  axs[0].set_xlabel("time (ps)")
  axs[0].set_ylabel('Potential Energy (kCal/mole)')

  axs[1].plot(reporters['Time (ps)'], reporters['Temperature (K)'])
  axs[1].set_xlabel("time (ps)")
  axs[1].set_ylabel('Temperature (K)')

  axs[2].plot(reporters['Time (ps)'], reporters['Density (gram/(item*milliliter))'])
  axs[2].set_xlabel("time (ps)")
  axs[2].set_ylabel('Density (g/mL)');

Now use it:

In [ ]:
plot_reporters('abl_ligand.log')

You will see how over the MD simulation time the system warms up and also compresses towards a more realistic density of about 1 g/ml.

## To recap:

1. You used the AMBER-derived *.inpcrd and *.prmtop files to create an OpenMM `system`.
2. You created an OpenMM `simulation` from this, additionally specifying things like the MD integrator to use.
3. You ran energy minimisation on the `simulation` object, then after attaching `reporters` to it, ran a short MD simulation.
4. You visualized the trajectory file using `mdtraj` and `nglview`, and system parameters from the log file using `pandas` and `matplotlib`.